In [18]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd
import csv


In [32]:
#Initialize the model
model = pyo.ConcreteModel("National Park Trip")

In [33]:
#Importing the data
activity_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Activities")
dist_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Distances")
dist_matrix = dist_matrix.drop(columns=["Unnamed: 0"])
time_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Travel Time") 


In [34]:
# Sets
#activity_table = activity_table.set_index("Activity")
model.I = pyo.Set(initialize=activity_table['Activity'].dropna().tolist()) #set of activities
#model.P = pyo.Set(initialize=['Hiking', 'Adventures', 'Shopping', 'Driving', 'Vistas']) # category of activities
model.K = pyo.Set(initialize=[1,2,3]) #set of days
model.L = pyo.Set() #set of lodging options from its own file
model.N = pyo.Set() #set of all nodes in network flow (lodging + activities)

In [35]:
#Parameters
duration_dict = activity_table.set_index("Activity")["Duration"].to_dict() #duration of each activity
activityType_dict = activity_table.set_index("Activity")["Category"].to_dict() #type of each activity
cost_dict = activity_table.set_index("Activity")["Cost"].to_dict() #cost of each activity
funPoints_dict = activity_table.set_index("Activity")["Fun Points"].to_dict() #fun points of each activity

distance_dict = {}
for i, src in enumerate(activity_table['Activity']):
    for j, dest in enumerate(activity_table['Activity']):
        if i != j:
            distance_dict[(src, dest)] = dist_matrix.iloc[i][j]
        else:
            distance_dict[(src, dest)] = 0

travel_time_dict = {}
for i, src in enumerate(activity_table['Activity']):
    for j, dest in enumerate(activity_table['Activity']):
        if i != j:
            travel_time_dict[(src, dest)] = time_matrix.iloc[i][j]
        else:
            travel_time_dict[(src, dest)] = 0

model.duration = pyo.Param(model.I, initialize=duration_dict)
model.activity_type = pyo.Param(model.I, initialize=activityType_dict, within=pyo.Any)
model.cost = pyo.Param(model.I, initialize=cost_dict)
model.fun_points = pyo.Param(model.I, initialize=funPoints_dict)



In [36]:
#Variables
model.x = pyo.Var(model.I, model.K, within=pyo.Binary) #activity selection 
model.y = pyo.Var(distance_dict.keys(), within=pyo.Binary) #path between locations (determines order)
model.h = pyo.Var(model.L, model.K, within=pyo.Binary) #lodging selection
model.v = pyo.Var(model.L, model.I, model.K, within=pyo.Binary) #if departing lodge L to activity i on day k 
model.w = pyo.Var(model.L, model.I, model.K, within=pyo.Binary) #if arriving at lodge L from activity i on day k






In [37]:
#Constraints
def hiking_time(model, k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Hiking') <= 2
model.hiking_time = pyo.Constraint(model.K, rule=hiking_time)

# def adventure_budget(model, k):
#     return sum(model.cost[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Adventures') <= 600
# model.adventure_budget = pyo.Constraint(model.K, rule=adventure_budget)


# def min_fun(model):
#     return sum(model.fun_points[i] * model.x[i, k] for i in model.I for k in model.K) >= 10
# model.min_fun = pyo.Constraint(rule=min_fun)   


#set fun points per day to 10
# def min_fun_per_day(model, k):
#     return sum(model.fun_points[i] * model.x[i, k] for i in model.I) >= 10
# model.min_fun_per_day = pyo.Constraint(model.K, rule=min_fun_per_day)


def min_activities(model):
    return sum(model.x[i, k] for i in model.I for k in model.K) >= 1
model.min_activities = pyo.Constraint(rule=min_activities)

# #Constraint to avoid selecting the same activity on different days
def same_activity(model, i):
    return sum(model.x[i, k] for k in model.K) <= 1
model.same_activity = pyo.Constraint(model.I, rule=same_activity)

# #Constraint activity time per day to 10 hours
def activity_time(model, k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I) <= 10

# #Constraint to avoid selecting the same activity on the same day
def same_day_activity(model, k):
    return sum(model.x[i, k] for i in model.I) <= 1
model.activity_time = pyo.Constraint(model.K, rule=activity_time)

# #Minimum number of activities per day
def min_activities_per_day(model, k):
    return sum(model.x[i, k] for i in model.I) >= 3
model.min_activities_per_day = pyo.Constraint(model.K, rule=min_activities_per_day)

# #Total length of a day inclusing traveling between activities needs to be less than 16 hours
# def total_time(model, k):
#     return sum(model.duration[i] * model.x[i, k] for i in model.I) + sum(model.y[i,j] * travel_time_dict[i,j] for i,j in distance_dict.keys()) <= 7




#def
'''
def scheduling_continutity(model, i):
    return sum(model.y[i,j] for j in model.N) == sum(model.x[i,p,k] for p in model.P for k in model.K)
model.scheduling_continuity = pyo.Constraint(model.I, rule=scheduling_continutity)
'''
'''
def travel_time(model):
    return sum(model.y[i,j] * r_ij[i,j] for i,j in model.N) <= sum(model.x[i,p,k] * t_ip[i] for i in model.I for p in model.P for k in model.K)
model.travel_time = pyo.Constraint(model.K, rule=travel_time)
'''

'\ndef travel_time(model):\n    return sum(model.y[i,j] * r_ij[i,j] for i,j in model.N) <= sum(model.x[i,p,k] * t_ip[i] for i in model.I for p in model.P for k in model.K)\nmodel.travel_time = pyo.Constraint(model.K, rule=travel_time)\n'

In [42]:
#Objective Function
def total_cost(model):
    #lodging_cost = sum(model.h[l, k] * activity_table.loc[l, 'Cost'] for l in model.L for k in model.K)
    activity_cost = sum(model.x[i,k] * activity_table.at[i, 'Cost'] for i in model.I for k in model.K)
    car_cost = sum(model.y[i,j] * travel_time_dict[i,j] for i,j in distance_dict.keys())

    return car_cost

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

'pyomo.core.base.objective.ScalarObjective'>) on block National Park Trip with
a new Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [43]:
#Solve the model
SOLVER.solve(model)
model.display()
#Extract the results
selected_activities = []
for i in model.I:
    for k in model.K:
        if pyo.value(model.x[i, k]) > 0:
            selected_activities.append(i)

#Print which activity is happening on which day 
fun_p = 0
for k in model.K:
    print(f"Day {k}:")
    for i in model.I:
        if pyo.value(model.x[i, k]) > 0:
            print(f"  - {activity_table.at[i, 'Name']} (Duration: {activity_table.at[i, 'Duration']} hours)")
            fun_p += activity_table.at[i, 'Fun Points']

    print()

print(f"Total Fun Points: {fun_p}")


RuntimeError: Solver does not currently have a valid solution. Please check the termination condition.